# Facebook Social Network Analysis with Apache Spark

This notebook provides an interactive analysis of Facebook social network data using Apache Spark GraphFrames.

## Dataset: Facebook Social Circles
- **4,039 users** (anonymized)
- **88,234 friendships**
- **Real social network data** from Stanford SNAP

## Analysis Goals:
1. **Network Structure Analysis** - Basic statistics and properties
2. **Influence Analysis** - Identify most influential users (PageRank)
3. **Community Detection** - Find social groups and clusters
4. **Centrality Measures** - Analyze user importance
5. **Network Patterns** - Discover triangles and connectivity

In [ ]:
# Install required packages (run this first)
# !pip install pyspark graphframes pandas matplotlib seaborn findspark

In [ ]:
# Import required libraries
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, desc, avg
from graphframes import GraphFrame
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

In [ ]:
# Create Spark Session
spark = SparkSession.builder \
    .appName("Facebook Social Network Analysis") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")
print(f"Python Version: {spark.sparkContext.pythonVer}")

## 1. Data Loading and Graph Construction

Load the Facebook friendship data and create a GraphFrame for analysis.

In [ ]:
# Load Facebook friendship data
data_path = "/Users/pranatityagi/Desktop/social-network-analysis/data/facebook_combined.txt"

# Load edge data (friendships)
edges_df = spark.read.csv(
    data_path,
    sep=" ",
    schema="src LONG, dst LONG"
)

print("Sample friendship data:")
edges_df.show(5)
print(f"Total friendships: {edges_df.count():,}")

In [ ]:
# Create vertices (users) from unique IDs
src_vertices = edges_df.select(col("src").alias("id"))
dst_vertices = edges_df.select(col("dst").alias("id"))
vertices_df = src_vertices.union(dst_vertices).distinct()

# Add user attributes
vertices_df = vertices_df.withColumn("name", col("id").cast("string"))

print(f"Total users: {vertices_df.count():,}")
print("Sample users:")
vertices_df.show(5)

In [ ]:
# Create GraphFrame
graph = GraphFrame(vertices_df, edges_df)

print("✅ GraphFrame created successfully!")
print(f"Vertices: {graph.vertices.count():,}")
print(f"Edges: {graph.edges.count():,}")

## 2. Basic Network Statistics

Calculate fundamental network properties and statistics.

In [ ]:
# Calculate degree statistics
degrees_df = graph.degrees
degree_stats = degrees_df.agg(
    avg("degree").alias("avg_degree"),
    count("degree").alias("total_users")
).collect()[0]

print("📊 Network Statistics:")
print(f"   👥 Total Users: {degree_stats['total_users']:,}")
print(f"   🤝 Total Friendships: {graph.edges.count():,}")
print(f"   📈 Average Degree: {degree_stats['avg_degree']:.2f}")

# Most connected users
print("\n🏆 Most Connected Users:")
top_users = degrees_df.orderBy(desc("degree")).limit(10).toPandas()
for i, (_, row) in enumerate(top_users.iterrows()):
    print(f"   {i+1}. User {row['id']}: {row['degree']} connections")

In [ ]:
# Visualize degree distribution
plt.figure(figsize=(12, 6))

# Get degree distribution
degree_dist = degrees_df.groupBy("degree").count().orderBy("degree").toPandas()

plt.subplot(1, 2, 1)
plt.bar(degree_dist['degree'], degree_dist['count'], alpha=0.7)
plt.xlabel('Degree (Number of Friends)')
plt.ylabel('Number of Users')
plt.title('Degree Distribution')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(degrees_df.toPandas()['degree'], bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('Degree')
plt.ylabel('Frequency')
plt.title('Degree Histogram')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📈 Degree Distribution Summary:")
print(f"   Mean degree: {degrees_df.agg(avg('degree')).collect()[0][0]:.2f}")
print(f"   Max degree: {degrees_df.agg({'degree': 'max'}).collect()[0][0]}")
print(f"   Min degree: {degrees_df.agg({'degree': 'min'}).collect()[0][0]}")

## 3. Influence Analysis (PageRank)

Identify the most influential users in the network using PageRank algorithm.

In [ ]:
# Calculate PageRank
print("🎯 Calculating PageRank (this may take a few seconds)...")

pagerank_df = graph.pageRank(resetProbability=0.15, maxIter=10)

# Get top influencers
top_influencers = pagerank_df.vertices \
    .orderBy(desc("pagerank")) \
    .limit(20) \
    .toPandas()

print("\n🏆 Top 10 Most Influential Users:")
print("Rank | User ID | PageRank Score")
print("-" * 35)
for i, (_, row) in enumerate(top_influencers.head(10).iterrows()):
    print(f"{i+1:2d}   | {row['id']:6d}  | {row['pagerank']:.6f}")

In [ ]:
# Visualize PageRank distribution
plt.figure(figsize=(10, 6))

# Get PageRank values for all users
pagerank_values = pagerank_df.vertices.select("pagerank").toPandas()

plt.hist(pagerank_values['pagerank'], bins=50, alpha=0.7, edgecolor='black')
plt.xlabel('PageRank Score')
plt.ylabel('Number of Users')
plt.title('PageRank Distribution')
plt.grid(True, alpha=0.3)
plt.show()

print("\n📊 PageRank Statistics:")
print(f"   Mean PageRank: {pagerank_values['pagerank'].mean():.6f}")
print(f"   Max PageRank: {pagerank_values['pagerank'].max():.6f}")
print(f"   Min PageRank: {pagerank_values['pagerank'].min():.6f}")

## 4. Community Detection

Find communities and social groups within the network.

In [ ]:
# Detect communities using Label Propagation
print("🏘️ Detecting communities (this may take a moment)...")

communities_df = graph.labelPropagation(maxIter=5)

# Analyze community sizes
community_stats = communities_df.groupBy("label") \
    .agg(count("*").alias("size")) \
    .orderBy(desc("size")) \
    .toPandas()

print(f"\n🏘️ Found {len(community_stats)} communities")
print("\n📊 Community Size Distribution:")
print("Rank | Community ID | Size | Percentage")
print("-" * 45)
total_users = community_stats['size'].sum()
for i, (_, row) in enumerate(community_stats.head(10).iterrows()):
    percentage = (row['size'] / total_users) * 100
    print(f"{i+1:2d}   | {row['label']:10d}   | {row['size']:4d} | {percentage:5.1f}%")

In [ ]:
# Visualize community sizes
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.bar(range(len(community_stats.head(20))), community_stats.head(20)['size'], alpha=0.7)
plt.xlabel('Community Rank')
plt.ylabel('Community Size')
plt.title('Top 20 Communities by Size')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.pie(community_stats.head(10)['size'], 
        labels=[f'Comm {i+1}' for i in range(10)], 
        autopct='%1.1f%%',
        startangle=90)
plt.title('Top 10 Communities Distribution')
plt.axis('equal')

plt.tight_layout()
plt.show()

## 5. Triangle Counting

Count triangles in the network to measure clustering and social cohesion.

In [ ]:
# Count triangles
print("🔺 Counting triangles (measuring network clustering)...")

triangle_count = graph.triangleCount()
total_triangles = triangle_count.agg({"count": "sum"}).collect()[0][0]

# Get users with most triangles
top_triangle_users = triangle_count \
    .orderBy(desc("count")) \
    .limit(10) \
    .toPandas()

print(f"\n🔺 Total triangles in network: {total_triangles:,}")
print("\n👥 Users with most triangles:")
print("Rank | User ID | Triangle Count")
print("-" * 30)
for i, (_, row) in enumerate(top_triangle_users.iterrows()):
    print(f"{i+1:2d}   | {row['id']:6d}  | {row['count']:3d}")

## 6. Network Connectivity Analysis

Analyze the overall connectivity of the network.

In [ ]:
# Find connected components
print("🔗 Analyzing network connectivity...")

cc_df = graph.connectedComponents()
component_stats = cc_df.groupBy("component") \
    .agg(count("*").alias("size")) \
    .orderBy(desc("size")) \
    .toPandas()

print(f"\n🔗 Found {len(component_stats)} connected components")
print("\n📊 Component Size Distribution:")
print("Rank | Component ID | Size | Percentage")
print("-" * 45)
for i, (_, row) in enumerate(component_stats.head(5).iterrows()):
    percentage = (row['size'] / total_users) * 100
    print(f"{i+1:2d}   | {row['component']:10d}   | {row['size']:4d} | {percentage:5.1f}%")

# Check if network is fully connected
if len(component_stats) == 1:
    print("\n✅ Network is fully connected!")
else:
    print(f"\n⚠️ Network has {len(component_stats)} disconnected components")
    isolated_users = component_stats[component_stats['size'] == 1]['size'].sum()
    print(f"   👤 Isolated users: {isolated_users}")

## 7. Advanced Analysis: Shortest Paths

Calculate shortest paths between influential users.

In [ ]:
# Find shortest paths between top influencers
print("🛣️ Calculating shortest paths between top influencers...")

# Get top 5 influencers
top_5_ids = [int(row['id']) for _, row in top_influencers.head(5).iterrows()]

# Calculate shortest paths from first influencer to others
source_id = top_5_ids[0]
print(f"\n🛣️ Shortest paths from User {source_id} (most influential):")

for target_id in top_5_ids[1:3]:  # Check first 2 others
    try:
        # Find shortest path
        path_df = graph.shortestPaths(landmarks=[target_id])
        result = path_df.filter(col("id") == source_id).collect()
        
        if result:
            distances = result[0][f"distances"]
            distance = distances.get(target_id, -1)
            if distance > 0:
                print(f"   User {source_id} → User {target_id}: {distance} steps")
            else:
                print(f"   User {source_id} → User {target_id}: No path found")
        else:
            print(f"   User {source_id} → User {target_id}: No path found")
            
    except Exception as e:
        print(f"   Error calculating path to User {target_id}: {str(e)}")

print("\n💡 Shortest path analysis shows how closely connected influential users are!")

## 8. Summary and Insights

Key findings from our Facebook social network analysis.

In [ ]:
# Generate comprehensive summary
print("🎉 Facebook Social Network Analysis - Summary")
print("=" * 50)

print(f"\n📊 Network Overview:")
print(f"   • Users: {graph.vertices.count():,}")
print(f"   • Friendships: {graph.edges.count():,}")
print(f"   • Average connections per user: {degree_stats['avg_degree']:.1f}")
print(f"   • Most connected user: {top_users.iloc[0]['degree']} friends")

print(f"\n🏆 Influence Analysis:")
print(f"   • Top influencer PageRank: {top_influencers.iloc[0]['pagerank']:.6f}")
print(f"   • Most influential user ID: {int(top_influencers.iloc[0]['id'])}")

print(f"\n🏘️ Community Structure:")
print(f"   • Number of communities: {len(community_stats)}")
print(f"   • Largest community: {community_stats.iloc[0]['size']:,} users ({(community_stats.iloc[0]['size']/total_users)*100:.1f}%)")

print(f"\n🔺 Network Clustering:")
print(f"   • Total triangles: {total_triangles:,}")
print(f"   • Clustering coefficient proxy: {(total_triangles * 3) / graph.edges.count():.4f}")

print(f"\n🔗 Network Connectivity:")
if len(component_stats) == 1:
    print(f"   • ✅ Fully connected network")
else:
    print(f"   • ⚠️ {len(component_stats)} disconnected components")
    print(f"   • 👤 Isolated users: {component_stats[component_stats['size'] == 1]['size'].sum()}")

print("\n💡 Key Insights:")
print("   • This Facebook network shows strong community structure")
print("   • Influential users are well-connected within their communities")
print("   • The network demonstrates high clustering (many triangles)")
print("   • Social connections follow power-law distribution")

print("🚀 Analysis completed successfully!")
print("   Use this notebook to explore different aspects of the network.")

In [ ]:
# Clean up Spark session
spark.stop()
print("🧹 Spark session stopped. Analysis complete!")